## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,45.91,56,59,11.34,broken clouds
1,1,Jamestown,US,42.0970,-79.2353,72.99,88,40,5.01,scattered clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,41.11,66,100,2.21,overcast clouds
3,3,East London,ZA,-33.0153,27.9116,52.81,69,0,12.37,clear sky
4,4,Georgetown,MY,5.4112,100.3354,82.89,89,20,2.30,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Jamestown,US,42.0970,-79.2353,72.99,88,40,5.01,scattered clouds
3,3,East London,ZA,-33.0153,27.9116,52.81,69,0,12.37,clear sky
4,4,Georgetown,MY,5.4112,100.3354,82.89,89,20,2.30,few clouds
5,5,Kapaa,US,22.0752,-159.3190,84.18,80,75,3.00,light rain
6,6,Deputatskiy,RU,69.3000,139.9000,53.13,55,33,7.56,scattered clouds
7,7,Puerto Ayora,EC,-0.7393,-90.3518,72.55,84,11,7.00,few clouds
9,9,Sept-Iles,CA,50.2001,-66.3821,62.26,92,40,5.01,scattered clouds
10,10,Mar Del Plata,AR,-38.0023,-57.5575,52.12,60,90,7.00,overcast clouds
12,12,Saint George,US,37.1041,-113.5841,85.53,55,20,4.00,few clouds
13,13,Aklavik,CA,68.2191,-135.0107,72.43,60,96,11.48,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Jamestown,US,42.0970,-79.2353,72.99,88,40,5.01,scattered clouds
3,3,East London,ZA,-33.0153,27.9116,52.81,69,0,12.37,clear sky
4,4,Georgetown,MY,5.4112,100.3354,82.89,89,20,2.30,few clouds
5,5,Kapaa,US,22.0752,-159.3190,84.18,80,75,3.00,light rain
6,6,Deputatskiy,RU,69.3000,139.9000,53.13,55,33,7.56,scattered clouds
7,7,Puerto Ayora,EC,-0.7393,-90.3518,72.55,84,11,7.00,few clouds
9,9,Sept-Iles,CA,50.2001,-66.3821,62.26,92,40,5.01,scattered clouds
10,10,Mar Del Plata,AR,-38.0023,-57.5575,52.12,60,90,7.00,overcast clouds
12,12,Saint George,US,37.1041,-113.5841,85.53,55,20,4.00,few clouds
13,13,Aklavik,CA,68.2191,-135.0107,72.43,60,96,11.48,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,72.99,scattered clouds,42.0970,-79.2353,
3,East London,ZA,52.81,clear sky,-33.0153,27.9116,
4,Georgetown,MY,82.89,few clouds,5.4112,100.3354,
5,Kapaa,US,84.18,light rain,22.0752,-159.3190,
6,Deputatskiy,RU,53.13,scattered clouds,69.3000,139.9000,
7,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,
9,Sept-Iles,CA,62.26,scattered clouds,50.2001,-66.3821,
10,Mar Del Plata,AR,52.12,overcast clouds,-38.0023,-57.5575,
12,Saint George,US,85.53,few clouds,37.1041,-113.5841,
13,Aklavik,CA,72.43,overcast clouds,68.2191,-135.0107,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!='']
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,72.99,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,East London,ZA,52.81,clear sky,-33.0153,27.9116,Tu Casa
4,Georgetown,MY,82.89,few clouds,5.4112,100.3354,Cititel Penang
5,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Puerto Ayora,EC,72.55,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,Sept-Iles,CA,62.26,scattered clouds,50.2001,-66.3821,Comfort Inn
10,Mar Del Plata,AR,52.12,overcast clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
12,Saint George,US,85.53,few clouds,37.1041,-113.5841,Best Western Plus Abbey Inn
13,Aklavik,CA,72.43,overcast clouds,68.2191,-135.0107,Aklavik Inn
14,Cockeysville,US,79.23,scattered clouds,39.4812,-76.6439,Embassy Suites by Hilton Baltimore Hunt Valley


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))